In [31]:
!pip install tinyec

In [32]:
!pip install pycryptodome

In [33]:
import pandas as pd
import pickle
# Descarga de librerías para la generación de llaves
from tinyec import registry
import secrets

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [6]:
dataset = pd.read_csv(r'C:\Users\Enrique\OneDrive - Instituto Tecnologico y de Estudios Superiores de Monterrey\Documents\Escuela\FJ23\Algebra moderna Cripto\reto\Reto-Git\LiCore-Crypto-IoT\Client_server\Prosumer_ABC.csv', header= 0, sep = ";" )
dataset.head(5)

,ID,Consumo (0) / Producción (1),Dia,Mes,Año,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,ABC,0,2,11,2013,58.00,75.0,65.0,0.08,67.0,...,338.0,369.0,318.00,322.0,345.0,298.00,316.00,299.0,341.00,304.0
1,ABC,0,3,11,2013,0.16,0.1,68.0,69.00,75.0,...,67.0,74.0,61.00,84.0,74.0,67.00,77.00,91.0,0.07,61.0
2,ABC,0,4,11,2013,0.08,66.0,77.0,64.00,84.0,...,114.0,106.0,128.00,511.0,74.0,84.00,59.00,69.0,69.00,71.0
3,ABC,0,5,11,2013,68.00,58.0,95.0,81.00,74.0,...,167.0,202.0,0.53,226.0,256.0,315.00,421.00,66.0,0.09,64.0
4,ABC,0,6,11,2013,77.00,61.0,97.0,65.00,78.0,...,117.0,345.0,351.00,69.0,57.0,0.07,0.07,72.0,56.00,78.0


In [ ]:
dataset['ID'].value_counts()

ABC    728
Name: ID, dtype: int64

In [ ]:
dataset.shape

(728, 101)

In [ ]:
import numpy as np
import hashlib

In [7]:
# algritmo euclideano extendido
# a - grupo
# b - elemento
def alg_euc_ext(a,b):
  if b == 0:
    d = a; x = 1; y =0
    return d,x, y

  x1 = 0; x2 = 1; y1 = 1; y2 = 0

  while b != 0:
    q = int(a/b)
    r = a - q*b
    x = x2 - q*x1
    y = y2 - q*y1
    a = b
    b = r
    x2 =x1
    x1 = x
    y2 = y1
    y1 = y

  d = a; x = x2; y = y2
  # y es inverso de a
  return d,x,y

def firmado_dsa(curve, m):
  # Generación de clave
  q = curve.field.n
  P = curve.g # element G of prime order q is chosen in E(GF(p))
  x = secrets.randbelow(q) #privkey
  Q = x * P # pubkey
  #h = hashlib.sha1(m)
  h = int(hashlib.sha512(m).hexdigest(),16)
  #Firma digital
  r = 0
  s = 0

  while r == 0 or s == 0:
    k = secrets.randbelow(q)
    kP = k * P
    r = kP.x % q
    l = alg_euc_ext(q, k)[2] # inverso de k
    s = (l*(h + x*r))%q
  return r, s, Q, h

def verificado_dsa(curve, r, s, Q, h):
  #Generaicón de clave
  q = curve.field.n
  P = curve.g

  #Verificación de la firma
  if 1<=r and 1<=s and r<=(q-1) and s<=(q-1):
    w = alg_euc_ext(curve.field.n, s)[2] # invserso s
    u1 = h * w % q
    u2 = r * w % q
    v = (u1*P + u2*Q).x%q
    if v == r:
      print('Se acepta la firma')
    else:
      print('Se rechaza la firma')

  else:
    print('Error: no cumple con 1<=r, s<=q-1')

### Comprobación RFC (ECDSA, 256 Bits (Prime Field))

In [ ]:
def firmado_dsa(curve, m , q, x, Ux, Uy, hash):
  # Generación de clave
  #q = curve.field.n
  P = curve.g # element G of prime order q is chosen in E(GF(p))
  #x = secrets.randbelow(q) #privkey
  Q = x * P # pubkey
  #h = hashlib.sha1(m)
  h = int(hash(m).hexdigest(),16)
  #Firma digital
  r = 0
  s = 0

  while r == 0 or s == 0:
    k = secrets.randbelow(q)
    kP = k * P
    r = kP.x % q
    l = alg_euc_ext(q, k)[2] # inverso de k
    s = (l*(h + x*r))%q
  return r, s, Q, h


In [23]:
import hmac
import hashlib

def deterministic_k(key, message):
    v = b'\x01' * 32
    k = b'\x00' * 32
    k = hmac.new(k, v + b'\x00' + message, hashlib.sha256).digest()
    v = hmac.new(k, v, hashlib.sha256).digest()
    k = hmac.new(k, v + b'\x01' + message, hashlib.sha256).digest()
    v = hmac.new(k, v, hashlib.sha256).digest()
    return int.from_bytes(v, 'big')

def firmado_dsa(curve, m, q, x, Ux, Uy, hash):
    P = curve.g
    #Q = x * P
    h = int(hash(m).hexdigest(), 16)

    r = 0
    s = 0

    while r == 0 or s == 0:
        k = deterministic_k(Ux, m.encode()) % q
        kP = k * P
        r = kP.x % q
        l = alg_euc_ext(q, k)[2]  # inverso de k
        s = (l * (h + x * r)) % q

    return r, s, k


In [24]:
q = 'FFFFFFFF00000000FFFFFFFFFFFFFFFFBCE6FAADA7179E84F3B9CAC2FC632551'
x = 'C9AFA9D845BA75166B5C215767B1D6934E50C3DB36E89B127B8A622B120F6721' # priv key
Ux = '60FED4BA255A9D31C961EB74C6356D68C049B8923B61FA6CE669622E60F29FB6'
Uy = '7903FE1008B8BC99A41AE9E95628BC64F2F1B20C2D7E9F5177A3C294D4462299'
m = "sample"
curve = registry.get_curve('brainpoolP256r1')

In [25]:
hash = lambda x: hashlib.sha1(x.encode())
firmado_dsa(curve, m, q, x, Ux, Uy, hash)

TypeError: unsupported operand type(s) for %: 'int' and 'str'

### Ejemplo con curva

In [ ]:
curve = registry.get_curve('brainpoolP256r1')
# Se convierte la base de datos a bytes para su manejo
m = pickle.dumps(dataset, protocol=4)

r,s,Q,h= firmado_dsa(curve, m)

verificado_dsa(curve, r, s, Q, h)

Se acepta la firma


### Por renglón

In [ ]:
# Firmado y lista con la curva, r, s y Q
def firmado_df(df):
  a = []
  for i in df.index:
    curve = registry.get_curve('brainpoolP256r1')
    m = pickle.dumps(df.iloc[i].to_numpy(), protocol=4)
    r,s,Q,h = firmado_dsa(curve, m)
    a.append([curve, r,s,Q,h])
  return a

In [ ]:
# Verificación firma
def ver_df(a):
  for i in range(len(a)):
    verificado_dsa(a[i][0],a[i][1],a[i][2],a[i][3],a[i][4])

In [ ]:
ver_df(firmado_df(dataset.iloc[0:5]))

Se acepta la firma
Se acepta la firma
Se acepta la firma
Se acepta la firma
Se acepta la firma


In [ ]:
dataset.iloc[0:1]

,ID,Consumo (0) / Producción (1),Dia,Mes,Año,1,2,3,4,5,...,87,88,89,90,91,92,93,94,95,96
0,ABC,0,2,11,2013,58.0,75.0,65.0,0.08,67.0,...,338.0,369.0,318.0,322.0,345.0,298.0,316.0,299.0,341.0,304.0


In [ ]:
firmado_df(dataset.iloc[0:5])

[[<tinyec.ec.Curve at 0x7f1fffd06dd0>,
  60089933663468299055621817546679137750978559245708831107792558496114058219345,
  34831935301576747237372464182751589532108630353834853593505119917594534237258,
  (70517755880371616239975555144258722646426394101001506394292817058863571689552, 43329819081636892630982966405395588075619846542826312892897683831502202431772) on "brainpoolP256r1" => y^2 = x^3 + 56698187605326110043627228396178346077120614539475214109386828188763884139993x + 17577232497321838841075697789794520262950426058923084567046852300633325438902 (mod 76884956397045344220809746629001649093037950200943055203735601445031516197751),
  1577118564501321564894959830782048992353602082558023345026291886523541253994903531347913055207288715772412847032037748353280786152445928468657328298591352],
  23728480287908685859706948613122796466828999032854313691216975562175034005259,
  26232009135395958837418550142026753708027063768876080799536299979071276070343,
  (6963056748614455052884675437650984